# COMP5318 Assignment 1: Rice Classification

##### Group number: A1 group-set1 177
##### Student 1 SID: 540660818
##### Student 2 SID: 550247340  

In [1]:
from sklearn.model_selection import StratifiedKFold

In [2]:
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

In [36]:
import pandas as pd
df = pd.read_csv("rice-final2.csv")

In [20]:
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

df.replace("?", np.nan, inplace=True)
for col in df.columns[:-1]:  
    df[col] = pd.to_numeric(df[col], errors='coerce')
imputer = SimpleImputer(strategy='mean')
df.iloc[:, :-1] = imputer.fit_transform(df.iloc[:, :-1])
scaler = MinMaxScaler()
df.iloc[:, :-1] = scaler.fit_transform(df.iloc[:, :-1])
df.iloc[:, -1] = df.iloc[:, -1].replace({"class1": 0, "class2": 1}).astype(int)
X = df.iloc[:, :-1].to_numpy() # Fitur
y = df.iloc[:, -1].to_numpy() # Class variable
def print_data(X, y, n_rows=10):
    for example_num in range(n_rows):
        print(",".join("{:.4f}".format(feature) for feature in X[example_num]), end=",")
        print(y[example_num])

print_data(X, y, n_rows=10)

0.4628,0.5406,0.5113,0.4803,0.7380,0.4699,0.1196,1
0.4900,0.5547,0.5266,0.5018,0.7319,0.4926,0.8030,1
0.6109,0.6847,0.6707,0.5409,0.8032,0.6253,0.1185,0
0.6466,0.6930,0.6677,0.5961,0.7601,0.6467,0.2669,0
0.6712,0.6233,0.4755,0.8293,0.3721,0.6803,0.4211,1
0.2634,0.2932,0.2414,0.4127,0.5521,0.2752,0.2825,1
0.8175,0.9501,0.9515,0.5925,0.9245,0.8162,0.0000,0
0.3174,0.3588,0.3601,0.3908,0.6921,0.3261,0.8510,1
0.3130,0.3050,0.2150,0.5189,0.3974,0.3159,0.4570,1
0.5120,0.5237,0.4409,0.6235,0.5460,0.5111,0.3155,1


### Part 1: Cross-validation without parameter tuning

In [21]:
from sklearn.model_selection import StratifiedKFold, cross_val_score

y = np.array(y, dtype=int)
cvKFold = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

In [22]:
from sklearn.linear_model import LogisticRegression
def logregClassifier(X, y):
    model = LogisticRegression(random_state=0, max_iter=1000)
    scores = cross_val_score(model, X, y, cv=cvKFold, scoring='accuracy')
    return scores.mean()

In [23]:
from sklearn.naive_bayes import GaussianNB

def nbClassifier(X, y):
    model = GaussianNB()
    scores = cross_val_score(model, X, y, cv=cvKFold, scoring='accuracy')
    return scores.mean()

In [24]:
from sklearn.tree import DecisionTreeClassifier

def dtClassifier(X, y):
    model = DecisionTreeClassifier(criterion="entropy", random_state=0)
    scores = cross_val_score(model, X, y, cv=cvKFold, scoring='accuracy')
    return scores.mean()

In [34]:
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier

# Bagging with Decision Trees
def bagDTClassifier(X, y, n_estimators=50, max_samples=1, max_depth=None):
    base_tree = DecisionTreeClassifier(criterion="entropy", max_depth=max_depth, random_state=0)
    model = BaggingClassifier(base_tree, n_estimators=n_estimators, max_samples=max_samples, random_state=0)
    scores = cross_val_score(model, X, y, cv=cvKFold, scoring='accuracy')
    return scores.mean()

# AdaBoost with Decision Trees
def adaDTClassifier(X, y, n_estimators=50, learning_rate=1, max_depth=1):
    base_tree = DecisionTreeClassifier(criterion="entropy", max_depth=max_depth, random_state=0)
    model = AdaBoostClassifier(base_tree, n_estimators=n_estimators, learning_rate=learning_rate, random_state=0)
    scores = cross_val_score(model, X, y, cv=cvKFold, scoring='accuracy')
    return scores.mean()

# Gradient Boosting
def gbClassifier(X, y, n_estimators=50, learning_rate=0.1):
    model = GradientBoostingClassifier(n_estimators=n_estimators, learning_rate=learning_rate, random_state=0)
    scores = cross_val_score(model, X, y, cv=cvKFold, scoring='accuracy')
    return scores.mean()

### Part 1 Results

In [35]:
# Running the classifiers and printing the results for Part 1

# Bagging
bag_n_estimators = 50
bag_max_samples = 100
bag_max_depth = 5

# AdaBoost
ada_n_estimators = 50
ada_learning_rate = 0.5
ada_max_depth = 5

# Gradient Boosting
gb_n_estimators = 50
gb_learning_rate = 0.5

logreg_score = logregClassifier(X, y)
nb_score = nbClassifier(X, y)
dt_score = dtClassifier(X, y)
bag_score = bagDTClassifier(X, y, n_estimators=bag_n_estimators, max_samples=0.8, max_depth=bag_max_depth)
ada_score = adaDTClassifier(X, y, n_estimators=ada_n_estimators, learning_rate=ada_learning_rate, max_depth=ada_max_depth)
gb_score = gbClassifier(X, y, n_estimators=gb_n_estimators, learning_rate=gb_learning_rate)

print(f"LR average cross-validation accuracy: {logreg_score:.4f}")
print(f"NB average cross-validation accuracy: {nb_score:.4f}")
print(f"DT average cross-validation accuracy: {dt_score:.4f}")
print(f"Bagging average cross-validation accuracy: {bag_score:.4f}")
print(f"AdaBoost average cross-validation accuracy: {ada_score:.4f}")
print(f"GB average cross-validation accuracy: {gb_score:.4f}")

LR average cross-validation accuracy: 0.9386
NB average cross-validation accuracy: 0.9264
DT average cross-validation accuracy: 0.9179
Bagging average cross-validation accuracy: 0.9400
AdaBoost average cross-validation accuracy: 0.9407
GB average cross-validation accuracy: 0.9321


### Part 2: Cross-validation with parameter tuning

In [27]:
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score

cvKFold = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)

In [28]:
# KNN
k = [1, 3, 5, 7]
p = [1, 2]


def bestKNNClassifier(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=0)
    param_grid = {'n_neighbors': k, 'p': p}
    knn = KNeighborsClassifier()
    grid_search = GridSearchCV(knn, param_grid, cv=cvKFold, scoring='accuracy')
    grid_search.fit(X_train, y_train)
    best_k = grid_search.best_params_['n_neighbors']
    best_p = grid_search.best_params_['p']
    best_cv_accuracy = grid_search.best_score_
    y_pred = grid_search.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_pred) 
    return best_k, best_p, best_cv_accuracy, test_accuracy


In [29]:
# SVM
C = [0.01, 0.1, 1, 5] 
gamma = [0.01, 0.1, 1, 10]

def bestSVMClassifier(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=0)
    param_grid = {'C': C, 'gamma': gamma}
    svm = SVC(kernel='rbf', random_state=0)
    grid_search = GridSearchCV(svm, param_grid, cv=cvKFold, scoring='accuracy')
    grid_search.fit(X_train, y_train)
    best_C = grid_search.best_params_['C']
    best_gamma = grid_search.best_params_['gamma']
    best_cv_accuracy = grid_search.best_score_
    y_pred = grid_search.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_pred)
    return best_C, best_gamma, best_cv_accuracy, test_accuracy

In [31]:
# Random Forest
n_estimators = [10, 30, 60, 100]
max_leaf_nodes = [6, 12]

def bestRFClassifier(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=0)
    param_grid = {'n_estimators': n_estimators, 'max_leaf_nodes': max_leaf_nodes}
    rf = RandomForestClassifier(criterion='entropy', max_features='sqrt', random_state=0)
    grid_search = GridSearchCV(rf, param_grid, cv=cvKFold, scoring='accuracy')
    grid_search.fit(X_train, y_train)
    best_n_estimators = grid_search.best_params_['n_estimators']
    best_max_leaf_nodes = grid_search.best_params_['max_leaf_nodes']
    best_cv_accuracy = grid_search.best_score_
    y_pred = grid_search.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_pred)
    test_macro_f1 = f1_score(y_test, y_pred, average='macro')
    test_weighted_f1 = f1_score(y_test, y_pred, average='weighted')
    
    return best_n_estimators, best_max_leaf_nodes, best_cv_accuracy, test_accuracy, test_macro_f1, test_weighted_f1

### Part 2: Results

In [32]:
# KNN
best_k, best_p, knn_cv_accuracy, knn_test_accuracy = bestKNNClassifier(X, y)
print("KNN best k:", best_k)
print("KNN best p:", best_p)
print("KNN cross-validation accuracy: {:.4f}".format(knn_cv_accuracy))
print("KNN test set accuracy: {:.4f}".format(knn_test_accuracy))
print()

# SVM
best_C, best_gamma, svm_cv_accuracy, svm_test_accuracy = bestSVMClassifier(X, y)
print("SVM best C:", best_C)
print("SVM best gamma:", best_gamma)
print("SVM cross-validation accuracy: {:.4f}".format(svm_cv_accuracy))
print("SVM test set accuracy: {:.4f}".format(svm_test_accuracy))
print()

# Random Forest
best_n_estimators, best_max_leaf_nodes, rf_cv_accuracy, rf_test_accuracy, rf_macro_f1, rf_weighted_f1 = bestRFClassifier(X, y)
print("RF best n_estimators:", best_n_estimators)
print("RF best max_leaf_nodes:", best_max_leaf_nodes)
print("RF cross-validation accuracy: {:.4f}".format(rf_cv_accuracy))
print("RF test set accuracy: {:.4f}".format(rf_test_accuracy))
print("RF test set macro average F1: {:.4f}".format(rf_macro_f1))
print("RF test set weighted average F1: {:.4f}".format(rf_weighted_f1))

KNN best k: 5
KNN best p: 1
KNN cross-validation accuracy: 0.9371
KNN test set accuracy: 0.9257

SVM best C: 5
SVM best gamma: 1
SVM cross-validation accuracy: 0.9457
SVM test set accuracy: 0.9343

RF best n_estimators: 30
RF best max_leaf_nodes: 12
RF cross-validation accuracy: 0.9390
RF test set accuracy: 0.9371
RF test set macro average F1: 0.9355
RF test set weighted average F1: 0.9370


### Part 3: Reflection

##### Write one paragraph describing the most important thing that you have learned throughout this assignment.
##### Student 1: Throughout this assignment, the most important thing I learned was the significance of model selection, hyperparameter tuning, and evaluation techniques in machine learning. By comparing different classifiers, I observed how each algorithm performs differently on the same dataset. This assignment underscored that no single model is universally best.
##### Student 2: The results are better if ensemble methods are used. This is because ensembles combine base classifiers and take the average results to increase accuracy.